Text classification using Roberta
=================================

We implemented text classification on the distaster tweets dataset and the imdb ranking dataset.
This second notebook will cover the imdb dataset.

Get the imdb dataset

In [ ]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar zxvf aclImdb_v1.tar.gz

Preprocess it

In [ ]:
!ls aclImdb/train

labeledBow.feat  pos	unsupBow.feat  urls_pos.txt
neg		 unsup	urls_neg.txt   urls_unsup.txt


Install libraries

In [ ]:
!pip install --upgrade transformers
!pip install simpletransformers

     |████████████████████████████████| 2.0MB 10.6MB/s 
     |████████████████████████████████| 3.2MB 41.2MB/s 
     |████████████████████████████████| 890kB 38.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=0da258882033ed1a9fd5100215f9fe060d25380d41a4e97c21c641654f396fff
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 215kB 10.1MB/s 
     |████████████████████████████████| 81kB 7.2MB/s 
     |████████████████████████████████| 1.2MB 18.7MB/s 
     |████████████████████████████████| 7.0MB 38.3MB/s 
     |████████████████████████████████| 194kB 43.3MB/s 
     |████████████████████████████████| 317kB 40.7MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
     |████████████████████████████████| 2.0MB 36.7MB/s 
     |████████████████████████████████| 163kB 35.9MB/s 
     |████████████████████████████

Imports

In [ ]:
from os import listdir
from os.path import join

import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel, ClassificationArgs

Load the dataset

In [ ]:
label_encode = {"neg":0, "pos": 1}


def load_imdb(path):
  labels = []
  texts = []
  for label in ["neg", "pos"]:
    for f in listdir(join(path, label)):
      with open(join(join(path, label), f)) as fp:
        texts.append(fp.read())
      labels.append(label_encode[label])
  return pd.DataFrame({"text": texts, "target": labels})

train = load_imdb("aclImdb/train")
test = load_imdb("aclImdb/test")
train

,text,target
0,Oh Geez... There are so many other films I wan...,0
1,"I was looking forward to this ride, and was ho...",0
2,The worst movie I have seen in a while. Yeah i...,0
3,You get a gift. It is exquisitely wrapped. The...,0
4,This film is really terrible. terrible as in i...,0
...,...,...
24995,I have been waiting for such an original pictu...,1
24996,I am watching the series back to back as fast ...,1
24997,The 1997 low-key indie dramedy Henry Fool woul...,1
24998,"If you enjoy the subtle (yes, I said subtle) a...",1


Split the labelled dataset into training and validation sets.

In [ ]:
train_df, valid_df = train_test_split(train, test_size=0.2, stratify=train["target"], random_state=0)

Define model

In [ ]:
model_args = ClassificationArgs(num_train_epochs=8, overwrite_output_dir=True, manual_seed=42)
model = ClassificationModel(model_type='roberta', model_name='roberta-base', use_cuda=True, num_labels=2, args=model_args)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

Train it

In [ ]:
model.train_model(train_df)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:449: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/20000 [00:00<?, ?it/s]

Epoch:   0%|          | 0/8 [00:00<?, ?it/s]

Running Epoch 0 of 8:   0%|          | 0/2500 [00:00<?, ?it/s]

Running Epoch 1 of 8:   0%|          | 0/2500 [00:00<?, ?it/s]

Running Epoch 2 of 8:   0%|          | 0/2500 [00:00<?, ?it/s]

Running Epoch 3 of 8:   0%|          | 0/2500 [00:00<?, ?it/s]

Running Epoch 4 of 8:   0%|          | 0/2500 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

Evaluate its performance

In [ ]:
result, model_outputs, wrong_preds = model.eval_model(valid_df)
print(result)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1029: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/5000 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/625 [00:00<?, ?it/s]

{'mcc': 0.7612197109172187, 'tp': 2118, 'tn': 2281, 'fp': 219, 'fn': 382, 'auroc': 0.95193864, 'auprc': 0.9518503254798096, 'eval_loss': 0.6716627651179209}


Predict on test set

In [ ]:
test_predictions, raw_outputs = model.predict(list(test.text.values))

  0%|          | 0/25000 [00:00<?, ?it/s]

  0%|          | 0/3125 [00:00<?, ?it/s]

Save as submission file

In [ ]:
test_original = pd.read_csv("/content/test.csv")

sample_sub = pd.DataFrame({"target": test_predictions, "id": test_original.id})
sample_sub.to_csv("submission.csv",index=False)
files.download("submission.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>